In [1]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


In [2]:
# # mount driver
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CardiacSeg/eda/Network
#
# #install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# !pip install ray
# %matplotlib inline

In [ ]:
# sync python module
%load_ext autoreload
%autoreload 2

import os
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.networkx.blocks.ham import HAM
from networks.ssl_head import SSLHead

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [64]:
# import torchvision.models as models
# import torch
# from ptflops import get_model_complexity_info


# macs, params = get_model_complexity_info(
#     net, 
#     (1, 128, 128, 128), 
#     as_strings=True,
#     print_per_layer_stat=False, 
#     verbose=False
# )
# print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
# print('{:<30}  {:<8}'.format('Number of parameters: ', params))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])

args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.1
args.kernel_size = 7
args.exp_rate = 4
args.norm_name = 'layer'
args.depths = [3, 3, 9, 3]
args.feature_size = 48         
args.use_init_weights = False
args.deep_sup=True
args.is_conv_stem=False
args.first_feature_size_half=False
args.skip_encoder_name=None #'convmod' #'eca'



net = None
del net
net = network('unetcnx_a1', args) # unetc2f
net.eval()

inp = None
del inp
inp = torch.rand((1, 1, args.roi_x, args.roi_y, args.roi_z)).to(args.device)
summary(net, inp)

# log_dir = '/nfs/Workspace/CardiacSeg/exps/exps/cotr/chgh/tune_results/t_4/main_0ef8c_00000_0_exp=exp_t_4_2023-04-02_10-06-38'
# model_pth = os.path.join(log_dir, 'models', 'final_model.pth')
# ckp = torch.load(model_pth)
# print('final early stop count:', ckp['early_stop_count'])
# print('final epoch:', ckp['epoch'])
# print('best val dice:', ckp['best_acc'])
# net.load_state_dict(ckp["state_dict"])

model: unetcnx_a1
patch size: 4
ker size: 7
exp rate: 4
feature sizes: [48, 96, 192, 384]
depths: [3, 3, 9, 3]
drop rate: 0.1
use init weights: False
is conv stem: False
use deep sup
                                                      Kernel Shape  \
Layer                                                                
0_encoder0.layer.conv1.Conv3d_conv                [1, 48, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                            -   
2_encoder0.layer.LeakyReLU_lrelu                                 -   
3_encoder0.layer.conv2.Conv3d_conv               [48, 48, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                            -   
5_encoder0.layer.conv3.Conv3d_conv                [1, 48, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                            -   
7_encoder0.layer.LeakyReLU_lrelu                                 -   
8_backbone.downsample_layers.0.Conv3d_0           [1, 48, 4, 4, 4]   
9_backbone.downsample_layers.0.LayerNorm_1     

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


Kernel Shape  \
Layer                                                             
0_encoder0.layer.conv1.Conv3d_conv             [1, 48, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                         -   
2_encoder0.layer.LeakyReLU_lrelu                              -   
3_encoder0.layer.conv2.Conv3d_conv            [48, 48, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                         -   
...                                                         ...   
174_decoder1.conv_block.InstanceNorm3d_norm2                  -   
175_decoder1.conv_block.conv3.Conv3d_conv     [96, 48, 1, 1, 1]   
176_decoder1.conv_block.InstanceNorm3d_norm3                  -   
177_decoder1.conv_block.LeakyReLU_lrelu                       -   
178_out_block.conv.Conv3d_conv                 [48, 2, 1, 1, 1]   

                                                       Output Shape   Params  \
Layer                                                                          
0_encoder0.layer.conv1.Conv3d_conv            [1, 48, 128, 128, 64]   1296.0   
1_encoder0.layer.InstanceNorm3d_norm1         [1, 48, 128, 128, 64]      NaN   
2_encoder0.layer.LeakyReLU_lrelu              [1, 48, 128, 128, 64]      NaN   
3_encoder0.layer.conv2.Conv3d_conv            [1, 48, 128, 128, 64]  62208.0   
4_encoder0.layer.InstanceNorm3d_norm2         [1, 48, 128, 128, 64]      NaN   
...                                                             ...      ...   
174_decoder1.conv_block.InstanceNorm3d_norm2  [1, 48, 128, 128, 64]      NaN   
175_decoder1.conv_block.conv3.Conv3d_conv     [1, 48, 128, 128, 64]   4608.0   
176_decoder1.conv_block.InstanceNorm3d_norm3  [1, 48, 128, 128, 64]      NaN   
177_decoder1.conv_block.LeakyReLU_lrelu       [1, 48, 128, 128, 64]      NaN   
178_out_block.conv.Conv3d_conv                 [1, 2, 128, 128, 64]     98.0   

                                                 Mult-Adds  
Layer                                                       
0_encoder0.layer.conv1.Conv3d_conv            1.358954e+09  
1_encoder0.layer.InstanceNorm3d_norm1                  NaN  
2_encoder0.layer.LeakyReLU_lrelu                       NaN  
3_encoder0.layer.conv2.Conv3d_conv            6.522982e+10  
4_encoder0.layer.InstanceNorm3d_norm2                  NaN  
...                                                    ...  
174_decoder1.conv_block.InstanceNorm3d_norm2           NaN  
175_decoder1.conv_block.conv3.Conv3d_conv     4.831838e+09  
176_decoder1.conv_block.InstanceNorm3d_norm3           NaN  
177_decoder1.conv_block.LeakyReLU_lrelu                NaN  
178_out_block.conv.Conv3d_conv                1.006633e+08  

[179 rows x 4 columns]

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.1
args.kernel_size = 7
args.exp_rate = 4
args.norm_name = 'layer'
args.depths = [4, 4, 4, 4]
args.feature_size = 48         
args.use_init_weights = False
args.deep_sup=True
args.is_conv_stem=True
args.skip_encoder_name='cbam'
args.deep_sup=True
net = None
del net
net = network('unetcnx_a1', args) # unetc2f
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))